In [176]:
import pandas as pd

In [177]:
df = pd.read_csv('nba_player_data.csv')

In [178]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [179]:
df['spread'] = df['team_score']-df['opp_score']

In [180]:
groups= df.groupby(['Game_Id','Team'],sort=False).sum()

In [181]:
matrix = groups.as_matrix()

In [182]:
ids = []
for game in df['Game_Id'].unique():
    ids.append(game+ '%aa'+game.split('_')[1])
    ids.append(game+ '%aa'+game.split('_')[2])


['10/25/2016_Cleveland_New York%aaCleveland', '10/25/2016_Cleveland_New York%aaNew York', '10/25/2016_Golden State_San Antonio%aaGolden State', '10/25/2016_Golden State_San Antonio%aaSan Antonio', '10/25/2016_Portland_Utah%aaPortland', '10/25/2016_Portland_Utah%aaUtah', '10/26/2016_Boston_Brooklyn%aaBoston', '10/26/2016_Boston_Brooklyn%aaBrooklyn', '10/26/2016_Indiana_Dallas%aaIndiana', '10/26/2016_Indiana_Dallas%aaDallas', '10/26/2016_LA Lakers_Houston%aaLA Lakers', '10/26/2016_LA Lakers_Houston%aaHouston', '10/26/2016_Memphis_Minnesota%aaMemphis', '10/26/2016_Memphis_Minnesota%aaMinnesota', '10/26/2016_Milwaukee_Charlotte%aaMilwaukee', '10/26/2016_Milwaukee_Charlotte%aaCharlotte', '10/26/2016_New Orleans_Denver%aaNew Orleans', '10/26/2016_New Orleans_Denver%aaDenver', '10/26/2016_Orlando_Miami%aaOrlando', '10/26/2016_Orlando_Miami%aaMiami']


In [183]:
new_frame = pd.DataFrame(matrix, columns = groups.columns, index = ids)

In [185]:
team_data = new_frame[['3P','3PA', 'AST', 'BLK', 'FG', 'FGA', 'FT', 'FTA', 'ORB',
       'PF', 'PTS', 'STL', 'TOV', 'TRB','is_Home','Team_Win']]

In [186]:
team_data = team_data.reset_index()

In [188]:
team_data['Team'] = team_data['index'].apply(lambda x: x.split('%aa')[1])

In [189]:
team_data['index'] = team_data['index'].apply(lambda x: x.split('%aa')[0])

In [190]:
team_data['is_Home'] = team_data['is_Home'].apply(lambda x: x>0).astype(int)
team_data['Team_Win'] = team_data['Team_Win'].apply(lambda x: x>0).astype(int)

In [191]:
team_data.head()

,index,3P,3PA,AST,BLK,FG,FGA,FT,FTA,ORB,PF,PTS,STL,TOV,TRB,is_Home,Team_Win,Team
0,10/25/2016_Cleveland_New York,9.0,27.0,17.0,6.0,32.0,87.0,15.0,20.0,13.0,22.0,88.0,6.0,18.0,42.0,0,0,Cleveland
1,10/25/2016_Cleveland_New York,13.0,35.0,31.0,5.0,45.0,94.0,14.0,19.0,11.0,22.0,117.0,12.0,14.0,51.0,1,1,New York
2,10/25/2016_Golden State_San Antonio,12.0,24.0,25.0,3.0,47.0,98.0,23.0,26.0,21.0,19.0,129.0,13.0,13.0,55.0,0,1,Golden State
3,10/25/2016_Golden State_San Antonio,7.0,33.0,24.0,6.0,40.0,85.0,13.0,18.0,8.0,19.0,100.0,11.0,16.0,35.0,1,0,San Antonio
4,10/25/2016_Portland_Utah,8.0,24.0,19.0,5.0,40.0,82.0,16.0,16.0,6.0,19.0,104.0,9.0,11.0,31.0,0,0,Portland


In [192]:
both_teams_total = team_data.groupby('index',sort=False).sum().reset_index()
##doing weird things. 
weird = pd.merge(both_teams_total,team_data,on='index')

In [197]:
#fill in opponent data in an insufficient way
team_data['opp_3P'] = weird['3P_x']-team_data['3P']
team_data['opp_3PA'] = weird['3PA_x']-team_data['3PA']
team_data['opp_AST'] = weird['AST_x']-team_data['AST']
team_data['opp_BLK'] = weird['BLK_x']-team_data['BLK']
team_data['opp_FG'] = weird['FG_x']-team_data['FG']
team_data['opp_FGA'] = weird['FGA_x']-team_data['FGA']
team_data['opp_FT'] = weird['FT_x']-team_data['FT']
team_data['opp_FTA'] = weird['FTA_x']-team_data['FTA']
team_data['opp_PTS'] = weird['PTS_x']-team_data['PTS']
team_data['opp_ORB'] = weird['ORB_x']-team_data['ORB']
team_data['opp_STL'] = weird['STL_x']-team_data['STL']
team_data['opp_PF'] = weird['PF_x']-team_data['PF']
team_data['opp_TOV'] = weird['TOV_x']-team_data['TOV']

In [202]:
def opponent(row):
    if row['Team'] == row['index'].split('_')[1]:
        return row['index'].split('_')[2]
    else:
        return row['index'].split('_')[1]

In [204]:
team_data['opponent'] =team_data.apply(opponent, axis=1)

In [211]:
team_data['Game_Id'] = team_data['index']
team_data.drop('index',axis=1, inplace=True)

In [213]:
team_data.to_csv('team_boxscores.csv')